In [1]:
!pip install ipython-sql
!pip install seaborn
import seaborn as sns
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("socioeconomic.db")
cur = con.cursor()
!pip install pandas 

In [3]:
%sql sqlite:///socioeconomic.db

In [1]:
import pandas as pd
import ssl
import sqlite3

#  Step 1: Disable SSL certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

#  Step 2: Load the CSV data from the HTTPS source
url = 'https://data.cityofchicago.org/resource/jcxq-k9xf.csv'
df = pd.read_csv(url)

#  Step 3: Create a SQLite database connection
con = sqlite3.connect("chicago_data.db")

#  Step 4: Save dataframe to SQL table
df.to_sql("chicago_socioeconomic_data", con, if_exists='replace', index=False, method="multi")

#  Step 5: Optional - verify by printing first few rows
print(df.head())


    ca community_area_name  percent_of_housing_crowded  \
0  1.0         Rogers Park                         7.7   
1  2.0          West Ridge                         7.8   
2  3.0              Uptown                         3.8   
3  4.0      Lincoln Square                         3.4   
4  5.0        North Center                         0.3   

   percent_households_below_poverty  percent_aged_16_unemployed  \
0                              23.6                         8.7   
1                              17.2                         8.8   
2                              24.0                         8.9   
3                              10.9                         8.2   
4                               7.5                         5.2   

   percent_aged_25_without_high_school_diploma  \
0                                         18.2   
1                                         20.8   
2                                         11.8   
3                                         13.4  

In [14]:
# Install the 'ipython-sql' and 'prettytable' libraries using pip
!pip install ipython-sql prettytable

# Import the 'prettytable' library, which is used to display data in a formatted table
import prettytable

# Set the default display format for prettytable to 'DEFAULT' (i.e., a simple table format)
prettytable.DEFAULT = 'DEFAULT'

In [16]:
%sql SELECT * FROM chicago_socioeconomic_data limit 5;

 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT * FROM chicago_socioeconomic_data limit 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [18]:
#How many rows are in the dataset?
%sql SELECT COUNT(*) FROM chicago_socioeconomic_data;

 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT COUNT(*) FROM chicago_socioeconomic_data;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [20]:
#How many community areas in Chicago have a hardship index greater than 50.0?
%sql SELECT COUNT(*) FROM chicago_socioeconomic_data WHERE hardship_index > 50.0;

 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT COUNT(*) FROM chicago_socioeconomic_data WHERE hardship_index > 50.0;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [22]:
#What is the maximum value of hardship index in this dataset?
%sql SELECT MAX(hardship_index) FROM chicago_socioeconomic_data;

 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT MAX(hardship_index) FROM chicago_socioeconomic_data;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [24]:
#Which community area which has the highest hardship index?

#We can use the result of the last query to as an input to this query:
%sql SELECT community_area_name FROM chicago_socioeconomic_data where hardship_index=98.0;

#or another option:
%sql SELECT community_area_name FROM chicago_socioeconomic_data ORDER BY hardship_index DESC LIMIT 1;

#or you can use a sub-query to determine the max hardship index:
%sql select community_area_name from chicago_socioeconomic_data where hardship_index = ( select max(hardship_index) from chicago_socioeconomic_data );

 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT community_area_name FROM chicago_socioeconomic_data where hardship_index=98.0;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT community_area_name FROM chicago_socioeconomic_data ORDER BY hardship_index DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: select community_area_name from chicago_socioeconomic_data where hardship_index = ( select max(hardship_index) from chicago_socioeconomic_data );]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [26]:
# Which Chicago community areas have per-capita incomes greater than $60,000?
%sql SELECT community_area_name FROM chicago_socioeconomic_data WHERE per_capita_income_ > 60000;

 * sqlite:///socioeconomic.db
(sqlite3.OperationalError) no such table: chicago_socioeconomic_data
[SQL: SELECT community_area_name FROM chicago_socioeconomic_data WHERE per_capita_income_ > 60000;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [7]:
# STEP 1: Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# STEP 2: Load the SQL extension if not already loaded
# %load_ext sql

# STEP 3: Connect to your SQLite database
# Make sure the connection string is correct for your environment
%sql sqlite:///chicago_data.db

# STEP 4: Run the SQL query to select the relevant columns
income_vs_hardship = %sql SELECT per_capita_income_, hardship_index FROM chicago_socioeconomic_data;

# STEP 5: Convert the SQL result to a DataFrame
df_plot = income_vs_hardship.DataFrame()

# STEP 6: Ensure the columns are numeric for plotting
df_plot['per_capita_income_'] = pd.to_numeric(df_plot['per_capita_income_'], errors='coerce')
df_plot['hardship_index'] = pd.to_numeric(df_plot['hardship_index'], errors='coerce')

# STEP 7: Drop any rows with NaN values after conversion
df_plot = df_plot.dropna()

# STEP 8: Plot the data using Seaborn
sns.set(style="whitegrid")
sns.jointplot(x='per_capita_income_', y='hardship_index', data=df_plot)

# Add a title to the plot
plt.suptitle('Per Capita Income vs Hardship Index', y=1.02)
plt.show()


 * sqlite:///chicago_data.db
Done.


KeyError: 'DEFAULT'